In [1]:
# 타이타닉 생존자 예측(이진분류)
from sklearn.metrics import accuracy_score, precision_score,recall_score,confusion_matrix

# 분류 모델의 평가 함수
def get_clf_eval(y_test,preds):
    confusion = confusion_matrix(y_test,preds) #혼동행렬(혼동행렬)의 값(실제,예측)
    accuracy = accuracy_score(y_test,preds) # 정확도
    precision = precision_score(y_test,preds) # 정밀도
    recall = recall_score(y_test,preds)#재현율
    
    print('오차행렬')
    print(confusion)
    print(f'정확도:{accuracy:.4f}, 정밀도: {precision:.4f}, 재현율:{recall:.4f}')

In [2]:
# 어제나갔던 데이터 전처리 -> 함수로 만들면 편하다 왜 ? 반복이 많다.
# 전처리에 관련된 함수 선언
# <전처리>
# 1.결측치 처리 , 2.필요없는 컬럼 삭제 , 3.인코딩
# [1].결측치 처리 함수: Age(평균), Cabin(N), Embarked(N) -> 데이터 프레임을 받아야 함

def fillna_(df): # df(dataframe, titanic)을 재료로 받겠다.
                # <주의>df가 복사되서 들어감 -> return 처리해야함
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    return df

# [2].필요 없는 컬럼 :PassengerId, Name, Ticket

def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)
    return df


# [3].레이블 인코딩 : Cabin(A~G,T) , Sex(M,F), Embarked(S,Q,C,N)
from sklearn.preprocessing import LabelEncoder #주의: import는 중간에서 X

def format_features(df):
    # Cabin(C80 ->C를 뻬애함)
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

def transform_features(df): # 이것만 실행하면 위에것1~3이 다 실행되는 함수
    df = fillna_(df)
    df = drop_features(df)
    df = format_features(df) # 최종적 df
    return df

# 파이프라인:데이터 전처리와 모델 학습 과정을 순차적으로 연결해 자동화하는 처리 흐름

In [3]:
# import 하는것도 이렇게 순서 지켜서 해야함
# <import 순서>: 커서한테 룰(rule) 줄때(린팅순서 시켜서 해줘~)
# [1] 내장모듈
# [2]설치된 모듈
# [3]서브파트 모듈

# lint(린트): 코드 품질검사/ 미국에서 린트안하면 짤림..바로
# 외국계회사는 린트 꼭 해야함

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # 분류모델
# 경고 강제 지우기
import warnings 
warnings.filterwarnings('ignore')



In [4]:
# 1.데이터 로딩
titanic_df = pd.read_csv(
    '../data/train.csv'
)
titanic_df.head()




,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# [1]가장중요 -전체 데이터에서 데이터와 레이블 분리
# 데이터만 전처리 한다!

y_titanic_df = titanic_df['Survived'] # label 추출
# inplace=False :원본에 반영하지 않는 대신 return은 해줘야함, 처리결과(DF)
X_titanic_df = titanic_df.drop('Survived', axis = 1, inplace=False)


In [6]:
# [2]이제서야 데이터 전처리
X_titanic_df = transform_features(X_titanic_df)
X_titanic_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,7,3
1,1,0,38.0,1,0,71.2833,2,0
2,3,0,26.0,0,0,7.9250,7,3
3,1,0,35.0,1,0,53.1000,2,3
4,3,1,35.0,0,0,8.0500,7,3


In [7]:
# 학습/테스트 데이트(레이블)분리
# 섞어서 추출한다
X_train,X_test,y_train,y_test = train_test_split(
    X_titanic_df,y_titanic_df,
    test_size=0.2,
    random_state=11    
    
)




In [8]:
# 모델링: 학습
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train) # 학습종료됨 : 모델 나옴
pred = lr_clf.predict(X_test) # 테스트 데이터(179)에 대한 예측값(179)
                              
print(pred)                              

[1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0
 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1
 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1 0 0
 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1]


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 333 to 703
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    int64  
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Cabin     712 non-null    int64  
 7   Embarked  712 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 50.1 KB


In [10]:
lr_clf =LogisticRegression(solver='liblinear')#솔버사용안하면 104 나옴
# 크로스 엔트로피 : 정보량, 엔트로피

In [11]:
lr_clf.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [12]:
# 학습종료 : 모델이 나온다.
pred = lr_clf.predict(X_test)
print(pred)
print(y_test.values) #이 값 집어넣어서 오차행렬 구한다.
# 위 -> 아래 칼럼 비교하기
# 1->1 살았는데 -> 살았다고 예측 : TP
# 0->1 죽었는데->살았다고  예측 : FN
# 0->0 죽었는데->죽었다고 예측 : TN

# 예측이 중요한거: 정밀도
# 실제가 중요한거: 재현율
# 결론) 다 예측을 기준으로 TP,TN,FP,FN 구하기

# <주의>P,F는 고정X, 비즈니스 목적따라 달라짐

[1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0
 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1
 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0
 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1]
[1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0
 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 1
 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0
 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0
 1 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1]


In [13]:
get_clf_eval(y_test,pred)
# 전체갯수: 179개

오차행렬
[[108  10]
 [ 14  47]]
정확도:0.8659, 정밀도: 0.8246, 재현율:0.7705


In [14]:
# 정확도 : 전체에서  T만 가져오기
# (108TN+47TP) / (108TN+47TP+10FP+14FN)
(108+47) / (108+14+10+47)

# 정밀도: 예측값 아래 방향으로 
# 정밀도는 위에서 아래로
47/(10+47)

# 재현율 
47/(14+47)

# 0(Negative)- 죽음
# 1(positive)- 삶
# TN(죽었다고 예측했는데 맞았다(실제죽음)0->0) : 108
# TP: 47
# FN: 14
# FP: 10

0.7704918032786885

In [15]:
# 4.정밀도-재현율 트레이드오프()
# 단)임계값 조정하는건 좋은건 아니다. 가능은 하지만~~
# 근데 조정하는이유? 결과값(0,1)을 바꿀려고
# ex) 임계값(0.5,기본값) 보다 크면 1
# ex) 임계값(0.5,기본값) 보다 작으면 0

# 근데 위에걸 알려면 예측확률 알아야 함
# 예측확률(0~1중간단계)을 뽑는 함수:predict_proba()
# ex)클래스 1일 확률를 뽑을 수 있다.


In [16]:
# 임계값 조정하기
# Binarizer : 숫자 데이터를 0과 1로 변환하는 전처리 도구
from sklearn.preprocessing import Binarizer 


In [17]:
# 결정 임계값(threshold)
from sklearn.preprocessing import Binarizer

# [1]예측 확률 구하기 : predict 보다 predict_proba가 먼저
lr_clf.predict_proba(X_test)
# 결과에서 오른쪽인 class[1]만 가져감
# 근데 0.55064773  -> 1나왔는데 .. 신뢰성이 떨어진다.....이거 잡고가야함


lr_clf.predict_proba(X_test)[:,1] >= 0.5
# 클래스가 1인! 놈의 확률이다
# [:,1]: 모든행의 두번째열 가져와사
# 1열의 모든행 가져오기,
#  >= 0.5  / 1->True , 0-> False

# [2] 클래스 1인 예측확률을  변수에 저장
pred_prod_1 = lr_clf.predict_proba(X_test)[:,1]

# 임계값 = 0.5
bina_05 = Binarizer(threshold=0.5)
pred_05 = bina_05.fit_transform(pred_prod_1.reshape(-1,1)) # 예측값 나오기
# 예측확률에서 예측값 뽑아내니까 transform
# <주의>fit_transform는 행렬이 들어가야함 -> 2차원들어가야함
# .reshape(-1,1) : 2차원으로 바꾸기
print('------임계값 0.5 일때 평가')
get_clf_eval(y_test, pred_05) # 모델의 분류 성능을 평가하는 사용자 정의 함수 호출

# 임계값 = 0.4
bina_04 = Binarizer(threshold=0.4)
pred_04 = bina_04.fit_transform(pred_prod_1.reshape(-1,1))
get_clf_eval(y_test,pred_04) # 재현율 올라가고 정확도 떨어짐

## <주의> 근데 이건 해석의 방법을 조정하는거당~~~


------임계값 0.5 일때 평가
오차행렬
[[108  10]
 [ 14  47]]
정확도:0.8659, 정밀도: 0.8246, 재현율:0.7705
오차행렬
[[97 21]
 [11 50]]
정확도:0.8212, 정밀도: 0.7042, 재현율:0.8197


In [18]:
# 함수:임계값변경해서 성능평가하는 함수
# ()에 재료는 3개 들어감
# <임계값 여러개 주기>
threshold = [0.4,0.45,0.5,0.55,0.6] # 임계값 여러개 주기

# [1]그리고 반복되니까~ 함수 만들기
def get_eval_of_threshold(y_test,pred_proba_,threshold):
    # bina = Binarizer(threshold = threshold).fit(pred_proba_)
    # custom_prdict = bina.transform(pred_proba_)
    bina = Binarizer(threshold = threshold).fit_transform(pred_proba_)
    # 원래 test data는 fit X하고transform만 해야하지만
    # test data랑 학습데이터같을경우엔 fit,transform 같이 상용가능
    
    # 주의: 종단 함수는 가공이 된놈(reshape)을 넣어야한다 .reshape()은 밖에서 해야함
    # ex)김치찌개만들때 다 잘라서(돼지고기,김치찌게) 넣어야함
    print(f'임계값 : {threshold}')
    get_clf_eval(y_test,bina)
    

threshold = [0.4,0.45,0.5,0.55,0.6]

# [2]함수만들고 반복
for t in threshold:
    get_eval_of_threshold(y_test,pred_prod_1.reshape(-1,1),t)

    
    # 위에서 받은놈으로 집어넣기
# 결론: 정밀도 올라가면 재현율 떨어진다. 재현율도 차이가 난다.정확도만 보면 안된다.

임계값 : 0.4
오차행렬
[[97 21]
 [11 50]]
정확도:0.8212, 정밀도: 0.7042, 재현율:0.8197
임계값 : 0.45
오차행렬
[[105  13]
 [ 13  48]]
정확도:0.8547, 정밀도: 0.7869, 재현율:0.7869
임계값 : 0.5
오차행렬
[[108  10]
 [ 14  47]]
정확도:0.8659, 정밀도: 0.8246, 재현율:0.7705
임계값 : 0.55
오차행렬
[[111   7]
 [ 16  45]]
정확도:0.8715, 정밀도: 0.8654, 재현율:0.7377
임계값 : 0.6
오차행렬
[[113   5]
 [ 17  44]]
정확도:0.8771, 정밀도: 0.8980, 재현율:0.7213


In [19]:
# 4.3 정밀도-재현율 곡선(Precision_Recall Curve)
# =곡선을 그릴수 있는 값을 만들어주는 함수
# = 임계값에 따른 precision , recall 값 구해주는 함수

# Curve라고 곡선이라고 생각하지 말것~~~



In [20]:
# 정밀도-재현율 곡선 그리기
from sklearn.metrics import precision_recall_curve

# 주의하기: precision_recall_curve -> arraylike 이기에 reshpae할 필요 없다.
# return:tuple[]
# 가공한놈을 넣어줘야한다.

# 클래스의 레이블이 1인 예측확률 추출
lr_clf.predict_proba(X_test) #2차원 나옴
lr_clf.predict_proba(X_test)[:,1] #[:,1]: 모든행의 두번째열 가져dhkfk

pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]
precisions,recalls,thresholds = precision_recall_curve(y_test,pred_proba_class1)




In [21]:


# 1차원,값들은 X축이다
print(f'반환된 분류 결정 임계값 배열의 shape:{thresholds.shape}') # 값:tick의 갯수
print(f'반환된 정밀도 배열의 shape:{precisions.shape}') 
print(f'반환된 재현율 배열의 shape:{recalls.shape}') 

# [0] :precision
# [1] : recall
# [2] : precision
 


반환된 분류 결정 임계값 배열의 shape:(165,)
반환된 정밀도 배열의 shape:(166,)
반환된 재현율 배열의 shape:(166,)


In [22]:
# 앞애서 5개 추출
print(f'임계값 5개 : {thresholds[:5]}')
print(f'정밀도 5개 : {precisions[:5]}')
print(f'재현율5개 : {recalls[:5]}')

임계값 5개 : [0.01974988 0.06956414 0.08402808 0.08474207 0.08920161]
정밀도 5개 : [0.34078212 0.34269663 0.34463277 0.34659091 0.34857143]
재현율5개 : [1. 1. 1. 1. 1.]


In [23]:
# 뒤에서 5개 추출
print(f'임계값 5개 : {thresholds[-5:]}')
print(f'정밀도 5개 : {precisions[-5:]}')
print(f'재현율5개 : {recalls[-5:]}')

# 그래프그리면 어디가 가장 좋은 임계값인지 찾을 수 있다.(중첩된곳)

임계값 5개 : [0.92638598 0.9283872  0.93261004 0.94040086 0.94326279]
정밀도 5개 : [1. 1. 1. 1. 1.]
재현율5개 : [0.08196721 0.06557377 0.03278689 0.01639344 0.        ]


In [25]:
# 202251103
# [1]데이터가 불균형이 인지 , 아닌지 먼저 파악하는게 중요
# [2]데이터 :불균형 -> 균형 맞추어야함
# [3]결론)정확도가 높은 모델이 우선!


# 평가지표: 내가 학습 시킨 모델이 학습이 다 끝난후 함.
# 중요: 오차가 적은놈이 우선, 애초에 F1 스코어 따지기 전에 오차적은 모델따지는게 중요함
# F1-Socre:안하는게 좋은거임, 애초에 오차작은모델이었다면 F1-score 할 필요 없음

# 산술평균:우리가 통계에 쓰는 평균
#   -문제점:정밀도:0.9 , 재현율:0.9 =>산술평균:정+/재/2(0.9) , 조화평균()
#   -가중치: 더 낮은 값에 패널티 부여
#   - 결론)산술평균은 조화롭지 못하다.
# 조화평균:조화로울수록 높게 나오는 평균 , 그래서 분산을 봐야함
# 분산이 작을수록 조화평균이 높아진다.


In [52]:
# f1 score
from sklearn.metrics import f1_score

f1 = f1_score(y_test,pred)
print(f'F1 Socre : {f1:.4f}') # F1 Socre : 0.7966 나오면 일단 망한거임......ㅋ_ㅋ


F1 Socre : 0.7966


In [53]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1=f1_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print(f'정확도:{accuracy:.4f},정밀도:{precision:.4f},재현율:{recall:.4f},F1:{f1:.4f}')




In [ ]:
# 임계값에 따른 정밀도와 재현율 조율한 값
# pred_proba_c1: class1의 예측확률
# thresholds: 임계값 리스트=>[0.4,0.45,0.5,0.55,0.6]
def get_eval_by_threshold(y_test,pred_proba_c1,thresholds): # ()안에는 확률값이 들어가야함
    for customer_threshold in thresholds:
        bina = Binarizer(threshold=customer_threshold).fit(pred_proba_c1)
        custom_predict = bina.transform(pred_proba_c1) #임계값에 따른 예측값 추출
        print(f"현재 임계값:{customer_threshold}")
        get_clf_eval(y_test,custom_predict) #현재 임계값에 따른 예측값
        
# Binarizer는 단수형 인자만 받음 / 마우스 갖다대서 파라미터 꼭 확인해라잉~
        

In [58]:
thresholds = [0.4,0.45,0.5,0.55,0.6]
pred_proba = lr_clf.predict_proba(X_test) #테스트 데이터에 대한 예측 확률
get_eval_by_threshold(y_test,pred_proba[:,1].reshape(-1,1),thresholds)

현재 임계값:0.4
오차행렬
[[97 21]
 [11 50]]
정확도:0.8212,정밀도:0.7042,재현율:0.8197,F1:0.7576
현재 임계값:0.45
오차행렬
[[105  13]
 [ 13  48]]
정확도:0.8547,정밀도:0.7869,재현율:0.7869,F1:0.7869
현재 임계값:0.5
오차행렬
[[108  10]
 [ 14  47]]
정확도:0.8659,정밀도:0.8246,재현율:0.7705,F1:0.7966
현재 임계값:0.55
오차행렬
[[111   7]
 [ 16  45]]
정확도:0.8715,정밀도:0.8654,재현율:0.7377,F1:0.7965
현재 임계값:0.6
오차행렬
[[113   5]
 [ 17  44]]
정확도:0.8771,정밀도:0.8980,재현율:0.7213,F1:0.8000


In [60]:
# ROC: 재현율 포함,설명, 시각화가 먼저, 역사적 유래때문에 재현율 강화할수 밖에 없음
# AUC: 수치화 먼저, AUC도 수치화 가능하긴함~~~,roc 아래 면적 곡선
#    - roc에 나온거를 정규화한거임,
#    - 많이 사용하게 됨
#    - 모델이 '양성' 과 '음성' 얼마나 잘 '구분'하는지를 평가하는 지표
# 진양성률(TPR): 실제 진짜인거 = 재현율, 실제를 얼마나 잘 예측하느냐 / (TPR = TP+FN) / 1일수록 좋음
# 위양성률(FPR): 실제 '음성'중에서 실제 양성인 부모, 틀린비율 / 0일수록 좋음

In [ ]:
# 파이썬 실습: ROC 곡선그리기 및 AUC 계산  -> 정량
# roc_curve() : 값을 얻기위한 함수, 직접 그리지는 X
# 'k--' : k:색갈은 검정, --:linestyle
#  

In [62]:
# AUC정리
# 1.임계값과 상관 X  -> 근본적인 성능 평가할때 사용함
# 2.데이터가 불균형한 경우에도 사용함

# AUC확장 : PR-AUC
# 불균형한 데이터일때 특화됨 
# <주의>불균형할때 정확도 보면 안됨


In [ ]:
# 종합 실습: 피마 인디언 당뇨병 예측
# 당뇨병 발병 여부(outcome) 예측하는 모델
# 과거Data --> 모델링(학습)

In [ ]:
# 데이터해석법
# 1.Data 가 불균형한지 , 불균형하면 AUC부터 봐야함

